In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests 
import PyPDF2
import re

In [3]:
articles = pd.read_csv('data/data.csv')
print(articles.head())

                                                data
0  named entity real world object assigned name e...
1  sql stand structured query language programmin...
2  although blockchain still infancy opportunity ...
3  job interview scary fresher especially attendi...
4  memcached highly performant distributed cachin...


In [4]:
data = list(articles['data'])
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

In [5]:
model = Doc2Vec(vector_size = 50,
min_count = 10,
epochs = 50
)

In [6]:
model.build_vocab(tagged_data)
k = list(model.wv.index_to_key)
print(len(k))

205


In [7]:
model.train(tagged_data,
total_examples = model.corpus_count,
epochs = model.epochs)
model.save('data/doc2vec.model')
print("Model saved")

Model saved


In [8]:
resume_path = 'data/resume1.pdf'
resume = ''
pdfReader = PyPDF2.PdfReader(resume_path)
for i in range(len(pdfReader.pages)):
    pageObj = pdfReader.pages[i]
    resume += pageObj.extract_text()

In [9]:
resume = resume.lower()
resume = re.sub('[^a-z]', ' ', resume)

In [10]:
resume

'vedansh shrivastavavedansh   gmail com                bhopal  dehradun education bachelor of technology  b tech   computer science with specialization in bigdata university of petroleum and energy studies             senior secondary  xii   science delhi public school  cbse board  year of completion       secondary  x  delhi public school  cbse board  year of completion       internships head of volunteers arushi ngo  bhopal may        jun      the internship was for handling the physically challenged students recording books for visually challenged students  trainings deep learning specialization deeplearning ai  online oct        nov      machine learning stanford university  coursera   online mar        may      apache spark with scala  hands on bigdata udemy  online feb        feb      projects face mask detection nov        nov      https   github com vedansh   face mask detection i used computer vison for this purpose by using pretrained network mobilenetv  and using mtcnn to ma

In [11]:
def extract_data(url):
    list1 = []
    count = 0
    resp = requests.get(url)
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text,'html.parser')
        l = soup.find(class_ = 'av-company-description-page mb-2')
        web = ''.join([i.text for i in l.find_all(['p', 'li'])])
        list1.append(web)
        return web
    else:
        print("Error")

In [12]:
jd_links = ['https://datahack.analyticsvidhya.com/jobathon/clix-capital/senior-manager-growthrisk-analytics-2',
'https://datahack.analyticsvidhya.com/jobathon/clix-capital/manager-growth-analytics-2',
'https://datahack.analyticsvidhya.com/jobathon/clix-capital/manager-risk-analytics-2',
'https://datahack.analyticsvidhya.com/jobathon/cropin/data-scientist-85']

In [13]:
jd_df = pd.DataFrame(columns = ['links', 'data'])
jd_df['links'] = jd_links

In [14]:
#Extracting the JD data from the links:
for i in range(len(jd_df)):
    jd_df['data'][i] = extract_data(jd_df['links'][i])

In [15]:
#Converting the text into lower case
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : str.lower(x))


In [16]:
#Removing the punctuations from the text
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : " ".join(re.findall('[\w]+',x)))

In [17]:
#Removing the numerics present in the text
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : re.sub(r'\d+','',x))

In [36]:
print(jd_df.data)

0    clix capital company overviewclix capital is a...
1    clix capital company overviewclix capital is a...
2    clix capital company overviewclix capital is a...
3    about uscropin at bangalore is an earth observ...
Name: data, dtype: object


In [22]:
model = Doc2Vec.load('data/doc2vec.model')
v1 = model.infer_vector(resume.split())
v2 = model.infer_vector(jd_df['data'][0].split())
cosine_similarity = (np.dot(np.array(v1), np.array(v2))) / ((np.linalg.norm(v1)) * (np.linalg.norm(v2)))
print(round(cosine_similarity, 3))

0.505
